<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
# Introduction to Databases: Cartesian Product, Union, Join, Inheritance 


## Some Helper Code Used in Cells in the Notebook


In [20]:
# We need a module (library) that can communicate with the
# database server.
import pymysql.cursors
import pandas as pd
import json


# The database server is running somewhere in the network.
# I must specify the IP address (HW server) and port number
# (connection that SW server is listening on)
# Also, I do not want to allow anyone to access the database
# and different people have different permissions. So, the
# client must log on.
config = {
  'user': 'dbuser',
  'password': 'dbuser',
  'host': '10.0.1.4',
  'database': 'lahman2016',
  'raise_on_warnings': True,
  'charset' : 'utf8'
}

# Connect
cnx = pymysql.connect(host='localhost',-
                             user='dbuser',
                             password='dbuser',
                             db='lahman2016',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

def run_query(q):
    print("Execution query = \n",q)
    with cnx.cursor() as cursor:
        # Create a new record
        cursor.execute(q)

    df_mysql = pd.read_sql(q,cnx)
    return df_mysql


def print_result(msg, pf):
    print(msg)
    print(pf)
    
def run_and_print_q(msg,q):
    r = run_query(q)
    print("\n",msg,"\n",r)
    
    
def run_query_to_d(q):
     with cnx.cursor() as cursor:
        # Create a new record
        cursor.execute(q)
        
     r = cursor.fetchall()
     return r


## Questions, Comments, Discussion

## Relational Continued



### Reminder
- There are two notations or representations of the algebra:
    - The original, formal theory.
    - SQL
    
__Original, Formal Notation__

- The "set" in the relational algebra is the set of _relations_.


- The operations are:
    - Common set operations:
        - Union: $\cup$
        - Intersection: $\cap$
        - Difference: $-$
    - Projection: $\pi$
    - Selection: $\sigma$
    - Cartesian Product: $\times$
    - Join: $\bowtie$
    - Rename/Alias
    

- The formal notation does not support create, update or delete. You could emulate the operations by
    - Defining new relations containing the created, updated or delete tuples.
    - Using $\cup$, $\cap$, $-$ on the original relation and created/deleted/updated tuple relation.
    
    
__SQL Notation__

- The "set" in the relational algebra is the catalog of _tables_.

- The operations are:
    - Common set operations:
        - Union: UNION
        - Intersection: INTERSECT
        - Difference: EXCEPT
    - Selection, Projection, Cartesian Product, Rename/Alias and Join are clauses within a SELECT statement.
    
    ```SELECT <project clause> FROM <table> [JOIN <table> [ON <join condition]] WHERE <select condition>```
    <br><br>
    
- SQL supports additional operations, e.g
    - GROUP BY
    - ORDER BY
    

### Cross-Product

- In set theory (and, usually, in other parts of mathematics), a Cartesian product is a mathematical operation that returns a set (or product set or simply product) from multiple sets. That is, for sets A and B, the Cartesian product A $\times$ B is the set of all ordered pairs $(a, b)$ where $a \in A$ and $b \in B$. 


- Cross-product/Cartesian product ($\times$) combines two relations.
    - If
        - Appetizers(dish)
        - Entrees(dish)
        - Desserts(dish)
    - We could say that $Meal(dish1, dish2, dish3)= Appetizers \times Entrees \times Desserts$
   
   
- The SQL syntaxt for Cartesian product is ```SELECT ... FROM A JOIN B```



In [12]:
run_and_print_q('Appetizers = ', "SELECT * FROM test.Appetizers;")
print("\n")
run_and_print_q('Entrees = ', "SELECT * FROM test.Entrees;")
print("\n")
run_and_print_q('Desserts = ', "SELECT * FROM test.Desserts;")
print("\n")
q = "SELECT test.Appetizers.dish as appetizer, " 
q = q + " test.Entrees.dish as entree, test.Desserts.dish as dessert  FROM test.Appetizers "
q = q + " JOIN test.Entrees JOIN test.Desserts;"
run_and_print_q("Meals = ", q)

Execution query = 
 SELECT * FROM test.Appetizers;

 Appetizers =  
        dish
0  Escargot
1     Salad
2    Shrimp


Execution query = 
 SELECT * FROM test.Entrees;

 Entrees =  
       dish
0  Chicken
1     Fish
2    Steak


Execution query = 
 SELECT * FROM test.Desserts;

 Desserts =  
         dish
0       Cake
1  Ice Cream


Execution query = 
 SELECT test.Appetizers.dish as appetizer,  test.Entrees.dish as entree, test.Desserts.dish as dessert  FROM test.Appetizers  JOIN test.Entrees JOIN test.Desserts;

 Meals =  
    appetizer   entree    dessert
0   Escargot  Chicken       Cake
1   Escargot  Chicken  Ice Cream
2      Salad  Chicken       Cake
3      Salad  Chicken  Ice Cream
4     Shrimp  Chicken       Cake
5     Shrimp  Chicken  Ice Cream
6   Escargot     Fish       Cake
7   Escargot     Fish  Ice Cream
8      Salad     Fish       Cake
9      Salad     Fish  Ice Cream
10    Shrimp     Fish       Cake
11    Shrimp     Fish  Ice Cream
12  Escargot    Steak       Cake
13  Esca

### Union, Intersection, Set Difference

The operators take two input relations, which must be _union-compatible:_
- Same number of fields.
- "Corresponding" fields have the same type.

__Union__

- $AllDishes = Appetizers \cup Entrees \cup Desserts$


- In SQL, the operator is $UNION$


In [19]:
q1 = "SELECT playerID, yearID, stint, G FROM Batting WHERE playerID='willite01'"
run_and_print_q("Season ", q1)
print("\n")
q2 = "SELECT playerID, yearID, round, G FROM BattingPost WHERE playerID='willite01'"
run_and_print_q("Post season ", q2)

q1 = "SELECT playerID, yearID, stint, 'R' as round, G FROM Batting WHERE playerID='willite01'"
q2 = "SELECT playerID, yearID, 0 as stint, round, G FROM BattingPost WHERE playerID='willite01'"
q = q1 + " UNION " + q2

run_and_print_q("All ", q)

Execution query = 
 SELECT playerID, yearID, stint, G FROM Batting WHERE playerID='willite01'

 Season  
      playerID  yearID  stint    G
0   willite01    1939      1  149
1   willite01    1940      1  144
2   willite01    1941      1  143
3   willite01    1942      1  150
4   willite01    1946      1  150
5   willite01    1947      1  156
6   willite01    1948      1  137
7   willite01    1949      1  155
8   willite01    1950      1   89
9   willite01    1951      1  148
10  willite01    1952      1    6
11  willite01    1953      1   37
12  willite01    1954      1  117
13  willite01    1955      1   98
14  willite01    1956      1  136
15  willite01    1957      1  132
16  willite01    1958      1  129
17  willite01    1959      1  103
18  willite01    1960      1  113


Execution query = 
 SELECT playerID, yearID, round, G FROM BattingPost WHERE playerID='willite01'

 Post season  
     playerID  yearID round  G
0  willite01    1946    WS  7
Execution query = 
 SELECT playerID, 

_Note_: 
1. You can use projection to "shape" incompatible tables.
1. Add columns the delineate from which table.
1. Combine, cast, convert, etc. columns.

__Intersection, Set Difference__

- Intersection ($\cap$) and Set Difference ($-$) have the standard set definitions.


- The SQL operations are INTERSECT and EXCEPT. 


- Some databases, including MySQL do not support INTERSECT and EXCEPT.


- These set operations are computable from JOIN and sub-queries.

| <img src="../images/L6_join_sets.jpg"> |
| --- |
| __Join and Set Operations__ |



### JOIN

#### General Concept

- Cross/Cartesian Product is the foundation for join $\bowtie$


- The _condition join_ or _theta join_ or $\theta-join$ is defined as

\begin{equation}
R \bowtie_C S = \sigma_C (R \times S)
\end{equation}


- I am not going to switch to showing results in SQL Workbench bench because all of the string and print stuff is driving me crazy.


- Consider $Master \times Batting$ --
    - That basically is a relation with every players record linked with every batting record, for every combination.
    - That does not make sense.
    - What is interesting is the subset of the cross product where the playerIDs match.
    
    
```
SELECT
	*
FROM
	Master
JOIN
	Batting
ON
	Master.playerID = Batting.playerID
```

- JOIN is significantly more efficient and intuitive than joining completely unrelated things and selecting out the stuff that does not make sense.

#### Some Special Cases

- An _Equi-Join_ is the special case in which the ON clause only checks equality.


- The _Natural Join_ is an Equi-Join on all common fields.

#### LEFT/RIGHT JOIN

Consider the query ...

```
SELECT
	* 
FROM 
	Master 
JOIN
	Batting
ON  
	batting.playerID = Master.playerID
ORDER BY
	Master.playerID ASC;
```

It produces ...

|<img src="../images/L6_player_season.jpeg"> |
| --- |
| __JOIN__ |


```
SELECT
	*
FROM
	Master
JOIN
	BattingPost
ON
	Master.playerID = BattingPost.playerID
ORDER BY
	Master.playerID ASC;
```

|<img src="../images/L6_player_post.jpeg"> |
| --- |
| __Dave Aardsma is Missing__ |


__Dave Aardsma is missing!__
- We know he played baseball.
- We isn't he in the JOIN?
- Because there is not post-season Batting tuple.
- JOIN only returns tuples where there are matches.



__LEFT (or RIGHT) JOIN__ handle the inclusion of tuples for which there is no match. The cross product will contain
    - The columns from the LEFT (or RIGHT) table
    - NULLS for the other columns.
    
```
SELECT
	* 
FROM 
	Master 
LEFT JOIN
	BattingPost
ON  
	battingpost.playerID = Master.playerID
ORDER BY
	Master.playerID ASC;
```

| <img src="../images/L6_left_join.jpeg">|
| --- |
| _LEFT JOIN Brings Back Dave_ |

### More Fun with Sets

#### Intersection

| <img src="../images/L6_intersection.jpeg">|
| --- |
| __Intersection__ |

- The first step is two ask two questions
    - What is the definition of being the same tuple? Primary keys match? All columns match? Some columns match?
    - How do you combine the columns with the same names that are not equal? Do you want the non-shared columns? In the following example,
        - How do you combine C, D?
        - Do you want E, F?
        
    
\begin{equation}
R(A,\ B,\ C,\ D)\ INNER\ JOIN\ S(A,\ B,\ C,\ D,\ E,\ F)\ ON\ R.A=S.A\ AND\ R.B=S.B 
\end{equation}
\begin{equation}
\rightarrow 
\end{equation}
\begin{equation}
(R.A,\ R.B,\ R.C,\ R.D,\ R.A,\ B,\ C,\ D,\ E,\ F).
\end{equation}

In [21]:
q21 = "SELECT * FROM Midterm.JaneEyre"
run_and_print_q("Words from Jane Eyre", q21)

Execution query = 
 SELECT * FROM Midterm.JaneEyre

 Words from Jane Eyre 
        Word  Count  id
0     all-1     19   1
1     all-2      7   2
2      jm-1      3   3
3      jm-2     17   4
4      jt-1      2   5
5      jt-2     17   6
6  j-only-1      6   7
7  j-only-2      9   8


In [22]:
q22 = "SELECT * FROM Midterm.MobyDick"
run_and_print_q("Words from Moby Dick", q22)

Execution query = 
 SELECT * FROM Midterm.MobyDick

 Words from Moby Dick 
        Word  Count  id
0     all-1    110   1
1     all-2     21   2
2      jm-1      3   3
3      jm-2     17   4
4      mt-1     13   5
5      mt-2      9   6
6  m-only-1      7   7
7  m-only-2     91   8


In [23]:
q23 = "SELECT * FROM Midterm.TomSawyer"
run_and_print_q("Words from Tom Sawyer", q23)

Execution query = 
 SELECT * FROM Midterm.TomSawyer

 Words from Tom Sawyer 
        Word  Count  id
0     all-1      7   1
1     all-2      7   2
2      jt-1      3   3
3      jt-2     17   4
4      mt-1      7   5
5      mt-2     11   6
6  t-only-1     13   7
7  t-only-2      3   8


- To simplify visual confirmation, there is a pattern on the words
    - all-* is in all 3 relations.
    - jt-* is in both Jane Eyre.
    - jm-* is in both Jane Eyre and Moby Dick.
    - mt-* is in both Moby Dick and Tom Sawyer.
    - t-only-* is only in Tom Sawyer, m-only-* for Moby Dick and j-only-* for Jane Eyre.
   
   
- ID does mean anything. It is a unused auto-increment column.


- count simulates the number of times the word appears in the book. How should the query combine the shared column with different values?

In [28]:
q31 = "SELECT Midterm.janeeyre.Word, Midterm.janeeyre.count + Midterm.mobydick.count as two_book_count, "
q31 = q31 + " Midterm.janeeyre.count as JaneEyre_count, Midterm.mobydick.count as MobyDick_count "
q31 = q31 + "FROM Midterm.janeeyre INNER JOIN Midterm.mobydick " 
q31 = q31 + "ON Midterm.janeeyre.word = Midterm.mobydick.word;"
run_and_print_q("Words shared between Moby Dick and Jane Eyre and the count totals", q31)

Execution query = 
 SELECT Midterm.janeeyre.Word, Midterm.janeeyre.count + Midterm.mobydick.count as two_book_count,  Midterm.janeeyre.count as JaneEyre_count, Midterm.mobydick.count as MobyDick_count FROM Midterm.janeeyre INNER JOIN Midterm.mobydick ON Midterm.janeeyre.word = Midterm.mobydick.word;

 Words shared between Moby Dick and Jane Eyre and the count totals 
     Word  two_book_count  JaneEyre_count  MobyDick_count
0  all-1             129              19             110
1  all-2              28               7              21
2   jm-1               6               3               3
3   jm-2              34              17              17


- The set operations are interesting but may seem artificial. When would you use them? Consider Master and AwardsPlayers


- Please show me information about players that have received an award since 2015. That is the intersection (INNER JOIN) of Master and AwardsPlayers on playerID.

In [29]:
q41 = "SELECT Master.playerID, Master.nameLast, master.nameFirst, AwardsPlayers.awardID, AwardsPlayers.yearID "
q41 += "FROM  Master JOIN AwardsPlayers ON Master.playerID = AwardsPlayers.playerID "
q41 += "WHERE AwardsPlayers.yearID > 2015;"
run_and_print_q("Players and Awards since 2015", q41)

Execution query = 
 SELECT Master.playerID, Master.nameLast, master.nameFirst, AwardsPlayers.awardID, AwardsPlayers.yearID FROM  Master JOIN AwardsPlayers ON Master.playerID = AwardsPlayers.playerID WHERE AwardsPlayers.yearID > 2015;

 Players and Awards since 2015 
      playerID   nameLast  nameFirst                              awardID  \
0   perezsa02      Perez   Salvador                       Silver Slugger   
1   cabremi01    Cabrera     Miguel                       Silver Slugger   
2   altuvjo01     Altuve       Jose                       Silver Slugger   
3   donaljo02  Donaldson       Josh                       Silver Slugger   
4   bogaexa01   Bogaerts     Xander                       Silver Slugger   
5   bettsmo01      Betts     Mookie                       Silver Slugger   
6   troutmi01      Trout       Mike                       Silver Slugger   
7   trumbma01     Trumbo       Mark                       Silver Slugger   
8   ortizda01      Ortiz      David             

- Which players _did not win an award_? That is the set difference $Master\ - \ AwardsPlayers.$

| <img src="../images/L6_fun_with_sets.jpeg">
| --- |
| __Fun with Sets__ |

- This combines _nerdy engineering and set theory_ with _baseball_ $\Rightarrow$ I can do this __all semester long.__


- This seems like obscure concepts, but does have applications.
    - Which customers have not purchased products for which I was running a marketing campaign?
    - Which products have _gold customers_ ordered that are also _on back order._
    
    
- These capabilities are useful in reporting, analytics, visualization, etc.


- I am not going to go through all of the various set operations and combining algebra in lectures.


- I will ask you to write some tricky set computations on the midterm and final.


- How do you do figure these tricky queries out? Decompose into more basic operations, and ...

| <img src="../images/L6_dear_google.jpeg"> |
| --- |
| __Dear Google__ |


- My focus in this course is teaching you the concepts and ensuring that you can apply in the real world.

\begin{equation}
Google \in RealWorld
\end{equation}


## Homework 2

### Status

- CAs and I are meeting at 1130 today to discuss.


- Improve clarity, validate scope and feasibility, identify grading objectives.


- Will publish final version later today on CourseWorks. There will be a pinned Piazza topic.

### Part 1 -- Conceptual Data Model

| <img src="../images/L5_HW_2_CM.jpeg"> | 
|:--:| 
|__HW 2 Conceptual Model__ |


### Logical and Physical Data Model

#### Overview

1. Download and install the full Lahman's 2016 Baseball Database SQL Version. The CAs and I will provide instructions and help.
<BR><BR>
1. HW2 only uses the tables above. Do not worry about the other tables for now.
<br><br>
1. Determine a _primary key_ for each table.
    1. Reminder
        1. A _super key_ is a combination of columns that uniquely identifies a row in a table.
        2. A _candidate key_ is a minimal super key.
        3. A _primary key_ is one of the candidate keys chosen by the data designer/modeler.
    1. I showed you how to determine candidate keys by trying queries. (Lecture 4)
    1. Modify the tables to add a _primary key._ You can use MySQL Workbench or some other tool. 
<br><br>
1. Determine a _foreign key_ relationship for each of the relationships in the diagram above. Create the foreign keys using MySQL Workbench or another tool. The foreign keys will be
    1. There will be two foreign keys in Appearances, Batting, Pitching, Fielding and Salaries.
    1. For each table, one foreign key references Master and one foreign key references Teams.


#### Worked Example



### Data Access Service API

#### API

- You will write a data access service API. For now, this will be a single function.

```
def dataservice_get(path,fields,filter):
    result = {}
    # Your code goes here.
    return result
```


- path can be of the form
    - /resource
    - /resource/metadata
    - /resource/primary-key
    - /resource/primary-key/resource
    
    
    
- If the path is of the form /resource/primary-key, /resource/primary-key/resource
    - _filter_ will be None
    - _fields_ is a list of fields to get from the identified resources.
    - The function returns a JSON object containing the identified resources and the selected properties.
    
    
    
- If path is of the form /resource
    - _filter_ is a SQL WHERE clause.
    - _fields_ is a list of columns to retrieve.
    - The function returns a JSON object containing the selected resources and the selected properties.
    
    
    
- If path is of the form /resource/metadata, the query returns a JSON object containing the information about the resources columns and keys.



- If the path contains a primary-key, the implementation uses "SHOW KEYS FROM resource" statement to
    - Convert the key in the path of the form k1-k2-k3
    - To the correct WHERE clause.



#### Examples

- Reource query
```
dataservice_get("/master",["nameLast","nameFirst","birthCountry"],"nameLast='Willis'")
```
returns
```
[
  {
      "nameLast": "Willis",
      "nameFirst": "Carl",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Dale",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Dontrelle",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Jim",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Joe",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Lefty",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Les",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Mike",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Ron",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Vic",
      "birthCountry": "USA"
   }
]
```

- /resource/primary-key: dataservice_get('/Batting/willite01-1940-1',[ '*' ] , None)
 

In [64]:
d=run_query_to_d("SHOW KEYS FROM Batting WHERE Key_name='PRIMARY'")
print("Batting keys = \n",json.dumps(d,indent=3))

Batting keys = 
 [
   {
      "Table": "batting",
      "Non_unique": 0,
      "Key_name": "PRIMARY",
      "Seq_in_index": 1,
      "Column_name": "playerID",
      "Collation": "A",
      "Cardinality": 18603,
      "Sub_part": null,
      "Packed": null,
      "Null": "",
      "Index_type": "BTREE",
      "Comment": "",
      "Index_comment": ""
   },
   {
      "Table": "batting",
      "Non_unique": 0,
      "Key_name": "PRIMARY",
      "Seq_in_index": 2,
      "Column_name": "yearID",
      "Collation": "A",
      "Cardinality": 93857,
      "Sub_part": null,
      "Packed": null,
      "Null": "",
      "Index_type": "BTREE",
      "Comment": "",
      "Index_comment": ""
   },
   {
      "Table": "batting",
      "Non_unique": 0,
      "Key_name": "PRIMARY",
      "Seq_in_index": 3,
      "Column_name": "stint",
      "Collation": "A",
      "Cardinality": 102492,
      "Sub_part": null,
      "Packed": null,
      "Null": "",
      "Index_type": "BTREE",
      "Comment": "",


    - Defines the key of Batting to be of the form playerID-yearID-stint
    
    - Which means the path /Batting/willite01-1940-1
    
    - Maps to the SQL Query
    
```
SELECT * FROM batting where playerid='willite01' AND yearID='1940' AND stint='1'
```

- The function call dataservice_get(/Master/willite01/Batting,['yearID','ab'],None) maps to the SQL statement
```
SELECT yearID, ab FROM Batting where playerID='willite01'
```


- The function call dataservice_get('/Master/metadata',None,None) returns

```
{
   "fields": [
      {
         "Field": "playerID",
         "Type": "varchar(255)",
         "Null": "NO",
         "Key": "PRI",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "yearID",
         "Type": "int(11)",
         "Null": "NO",
         "Key": "PRI",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "stint",
         "Type": "int(11)",
         "Null": "NO",
         "Key": "PRI",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "teamID",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "lgID",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "G",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "AB",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "R",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "H",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "2B",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "3B",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "HR",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "RBI",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "SB",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "CS",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "BB",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "SO",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "IBB",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "HBP",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "SH",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "SF",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "GIDP",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      }
   ],
   "keys": [
      {
         "Table": "batting",
         "Non_unique": 0,
         "Key_name": "PRIMARY",
         "Seq_in_index": 1,
         "Column_name": "playerID",
         "Collation": "A",
         "Cardinality": 18603,
         "Sub_part": null,
         "Packed": null,
         "Null": "",
         "Index_type": "BTREE",
         "Comment": "",
         "Index_comment": ""
      },
      {
         "Table": "batting",
         "Non_unique": 0,
         "Key_name": "PRIMARY",
         "Seq_in_index": 2,
         "Column_name": "yearID",
         "Collation": "A",
         "Cardinality": 93857,
         "Sub_part": null,
         "Packed": null,
         "Null": "",
         "Index_type": "BTREE",
         "Comment": "",
         "Index_comment": ""
      },
      {
         "Table": "batting",
         "Non_unique": 0,
         "Key_name": "PRIMARY",
         "Seq_in_index": 3,
         "Column_name": "stint",
         "Collation": "A",
         "Cardinality": 102492,
         "Sub_part": null,
         "Packed": null,
         "Null": "",
         "Index_type": "BTREE",
         "Comment": "",
         "Index_comment": ""
      }
   ]
}
```
